I noticed that when I used EarlyStop, it actually stopped the improvements of metrics.
In this notebook I will try the combination of two techniques:
- Dropout-layers
- ReduceLROnPlateau

## Uploading data, X-y variables, Train/test/validation split

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics

import tensorflow as tf
import keras
from keras import layers

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [2]:
# upload data
df = pd.read_csv("balanced_data.csv")

In [3]:
# leave out the target variable! (dependent variable)
X = df[['Year', 'Mileage', 'City', 'State', 'Make', 'Model']]

# have only the target variable here (dependent variable)
y = df['Price']

In [4]:
# first, train/test split => 70% for training, 30% for other purposes (temp)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=101)

# now, split the 30% for other purposes by 50% (resulting in 2 x 15%)
# so finally, we have:
# 70% for training
# 15% for testing
# 15% for validation
# => 70 + 15 +15 = 100%
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=101)

## Development of the neural network

In [ ]:
# Number of input features (after dropping the target column)
variable_amount = len(X.columns)

# ModelCheckpoint to save the best model based on validation loss
mc = ModelCheckpoint('best_model_regression1.keras', monitor='val_loss', mode='min', save_best_only=True)

# ReduceLROnPlateau to reduce learning rate when the validation loss has stopped improving
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, verbose=1)

# Combine all active callbacks into a list (excluding EarlyStopping)
callback_list = [mc, rlr]

# Define Sequential neural network model with Dropout layers
model = keras.Sequential(
    [
        layers.BatchNormalization(input_shape=(variable_amount,)),
        layers.Dense(16, activation="relu", kernel_regularizer=keras.regularizers.l1(l1=0.1)),
        layers.Dropout(0.1),
        layers.Dense(8, activation="relu"),
        layers.Dense(1)  # Output layer for regression (1 node, no activation function)
    ]
)

# Compile the model with Adam optimizer and Mean Squared Error loss function
model.compile(optimizer='adam', loss='mse')

# Print out the summary of the model
model.summary()

# Train the model with the dataset and apply the callbacks
history = model.fit(x=X_train, y=y_train, epochs=800, validation_data=(X_val, y_val), callbacks=callback_list)

/Users/emiliiazemskova/anaconda3/lib/python3.10/site-packages/keras/src/layers/normalization/batch_normalization.py:143: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 6)              │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 281 (1.10 KB)

 Trainable params: 269 (1.05 KB)

 Non-trainable params: 12 (48.00 B)

Epoch 1/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 393us/step - loss: 238970688.0000 - val_loss: 33088866.0000 - learning_rate: 0.0010
Epoch 2/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 380us/step - loss: 36208860.0000 - val_loss: 32980088.0000 - learning_rate: 0.0010
Epoch 3/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 384us/step - loss: 35973392.0000 - val_loss: 32976336.0000 - learning_rate: 0.0010
Epoch 4/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 380us/step - loss: 36180184.0000 - val_loss: 33043142.0000 - learning_rate: 0.0010
Epoch 5/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 395us/step - loss: 36116280.0000 - val_loss: 32972166.0000 - learning_rate: 0.0010
Epoch 6/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 398us/step - loss: 36021608.0000 - val_loss: 32974010.0000 - learning_rate: 0.0010
Epoch 7/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 370us/step - loss: 36066248.0000 - val_loss: 32976988.0000 - learning_rate: 0.0010
Epoch 8/800
4596/4596 ━━━━━━━━━━━━━━━━━━━━ 2s 367us/step - loss: 36100348.0000 - val_loss: 33063

## Error metrics

In [ ]:
# training and validation loss over epochs 
loss_df = pd.DataFrame(model.history.history)
loss_df.plot()

# insights:


In [ ]:
# compare the final model loss/evaluation values
print("Test data evaluation:")
print(model.evaluate(X_test, y_test, verbose=0))
print("\nTrain data evaluation:")
print(model.evaluate(X_train, y_train, verbose=0))

# the model is often good when these error values are similar


In [ ]:
test_predictions = model.predict(X_test)

# reshape the data for easier comparison table
test_predictions = pd.Series(test_predictions.reshape(len(y_test),))
pred_df = pd.DataFrame(np.asarray(y_test), columns=['Test True Y'])
pred_df = pd.concat([pred_df, test_predictions], axis=1)
pred_df.columns = ['Test True Y', 'Model Predictions']

# print the comparison table - true values vs. model predicted values
# we can nicely see here how far off our model is in some cases
pred_df

# insights:


In [ ]:
# these values follow a linear line = good predictions
# we basically compare the predicted values 
# to true test values and see the differences
sns.scatterplot(x='Test True Y', y='Model Predictions', data=pred_df)

# the same picture as in the original model

In [ ]:
# MAE - Mean average error
print("MAE")
print(round(metrics.mean_absolute_error(y_test, test_predictions), 2), "$")

# MSE - Mean square error
print("\nMSE")
print(round(metrics.mean_squared_error(y_test, test_predictions), 2), "$^2")

# RMSE - Root mean square error
print('\nRMSE:')
print(round(np.sqrt(metrics.mean_squared_error(y_test, test_predictions)), 2), "$")

# R-squared. 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
print('\nR-squared:')
print(round(metrics.r2_score(y_test, test_predictions), 2))

# Explained Variance Score => 0 = the model descibes the dataset poorly
# 1 = model describes the dataset perfectly
# high variance score = model is a good fit for the data 
# low variance score = model is not a good fit for the data
# the higher the score, the model is more able to explain the variation in the data
# if score is low, we might need more and better data
print("\nExplained variance score:")
print(round(metrics.explained_variance_score(y_test, test_predictions), 2))


# MAE
# Lower is better
# Original is 4841.92 $
# Now is 
# almost the same result

# MSE
# Lower is better
# Original is 34666769.92 $^2
# Now is 
# almost the same

# RMSE
# Lower is better
# Original is 5887.85 $
# Now is 
# a little difference

# R-squared
# Closer to 1 is better
# Original is 0.07
# Now is
# here we have a bit better results


# Explained variance score
# Closer to 1 is better
# Original is 0.1
# Now is 

# To sum up,